In [ ]:
from multiprocessing import Pool
import tqdm
from function_pontius import *
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Pontius metrics in this notebook:
1. Overall Difference (OD)
1. Overal Quantity Component (OQC)
1. Overall Allocation Component (OAC)
1. Overal Quantity Disagreement 'Nature' (OQD_n) --> Note that this is a categorical metric calculation

Using the contingency table from the previous metrics this notebook also calculates:
1. Overall Accuracy (OA)

In [2]:
# Assign focal land use class used for OQD_n; defaults to 'Nature' (class_id=22)
class_id = 22

# Lists to store the map and metric return values
maps1 = []
maps2 = []
oa = []
od = []
oqd = []
oad = []
oqd_n = []

#number of workers to use
num_workers = 12

with Pool(num_workers) as p:
    iterable =  [(i, class_id) for i in range(multi_its)]
    for n1, n2, acc, diff, qd, ad, qd_n in tqdm.tqdm(p.istarmap(calc_multi, iterable),
                       total=len(iterable)):
        maps1.append(n1)
        maps2.append(n2)
        oa.append(acc)
        od.append(diff)
        oqd.append(qd)
        oad.append(ad)
        oqd_n.append(qd_n)

100%|████████████████████████████████████████████████████████████████████████████████| 190/190 [05:18<00:00,  1.68s/it]


In [13]:
multi_df(maps1, maps2, oa, 'overallaccuracy')
multi_df(maps1, maps2, od, 'overalldifference')
multi_df(maps1, maps2, oqd, 'overallquantitydifference')
multi_df(maps1, maps2, oad, 'overallallocationdifference')
#Change used name based on the landuse class analyzed
df_id = 'quantitydifferencecategorical_' + str(class_id)
multi_df(maps1, maps2, oqd_n, df_id)

# Old code for comparison

In [ ]:
#import function_pontius_old as fpont
#from sklearn.metrics import confusion_matrix

In [ ]:
# Assign focal land use class used for OQD_n; defaults to 'Nature' (class_id=22)
class_id = 22

# Lists to store the map and metric return values
maps11 = []
maps22 = []
oa1 = []
od1 = []
oqd1 = []
oad1 = []
oqd_n1 = []

#number of workers to use
num_workers = 12

with Pool(num_workers) as p:
    iterable =  [(i, class_id) for i in range(multi_its)]
    for n1, n2, acc, diff, qd, ad, qd_n in tqdm.tqdm(p.istarmap(fpont.calc_multi, iterable),
                       total=len(iterable)):
        maps11.append(n1)
        maps22.append(n2)
        oa1.append(acc)
        od1.append(diff)
        oqd1.append(qd)
        oad1.append(ad)
        oqd_n1.append(qd_n)

# Compare old vs new code output

In [ ]:
oa = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overallaccuracy_df.csv')
oad = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overallallocationdifference_df.csv')
od = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overalldifference_df.csv')
oqd = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overallquantitydifference_df.csv')
oqd_n2 = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/quantitydifferencecategorical_22_df.csv')

In [7]:
oa2 = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overallaccuracy_df.csv')
oad2 = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overallallocationdifference_df.csv')
od2 = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overalldifference_df.csv')
oqd2 = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/overallquantitydifference_df.csv')
oqd_n2 = pd.read_csv('C:/LUMOS/MCK/output_DFs/compare/quantitydifferencecategorical_22_df.csv')

In [12]:
len(oa2)

20

In [9]:
oa == oa2

ValueError: Unable to coerce to Series, length must be 20: given 190

# Single Tests

In [ ]:
def calc_multi(pair_id, class1=22):
    """This function is the basis for calculating Pontius' metrics (2014), starting off by computing the confusion matrix for two provided maps and 
    ending with the Total Disagreement if it is assigned to be calculated."""
    map1 = pontius_pairs[pair_id][0]
    map2 = pontius_pairs[pair_id][1]
    m1_dir = abs_dir + 'ascmaps/' + map1 + '.asc'
    m2_dir = abs_dir + 'ascmaps/' + map2 + '.asc'
    m1 = np.loadtxt(m1_dir, skiprows=6)
    m2 = np.loadtxt(m2_dir, skiprows=6)
    #compute confusion matrix
    cm = conf_mat(m1, m2)
    # turn array into df so that ppy can work with it
    rows_cols = ['class' + str(i) for i in range(28)]
    rows_cols.remove('class23')
    rows_cols.remove('class27')
    df = pd.DataFrame(data=cm, index=rows_cols, columns=rows_cols)
    #create object to run ppy code
    df_pont = pont.pontiPy(df)
    return df_pont

In [ ]:
dfpont = calc_multi(0)

In [ ]:
dfpont.matrix()

In [ ]:
dfpy = calc_multi(0)

In [ ]:
dfpy.matrix()

In [ ]:
map1 = np.loadtxt('C:/LUMOS/MCK/ascmaps/map0.asc', skiprows=6)
map2 = np.loadtxt('C:/LUMOS/MCK/ascmaps/map1.asc', skiprows=6)
map1 = map1.astype('int32')
map2 = map1.astype('int32')
x1 = np.concatenate(map1, axis=0)
x2 = np.concatenate(map2, axis=0)

In [ ]:
dfpy

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array=dfpy

df_cm = pd.DataFrame(array)
plt.figure(figsize=(14,14))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, linewidths=1) # font size

plt.show()

In [ ]:
plt.figure(figsize=(14,14))
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()

In [ ]:
rows_cols = ['cat' + str(i + 1) for i in range(28)]

In [ ]:
df1 = pd.DataFrame(data=cm1, index=rows_cols, columns=rows_cols)
df2 = pd.DataFrame(data=cm2, index=rows_cols, columns=rows_cols)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

# import some data to play with
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01).fit(X_train, y_train)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(classifier, X_test, y_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

plt.show()